<a href="https://colab.research.google.com/github/ashalem/ML_Human/blob/main/HW4_W2024_students_517f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<div>Machine Learning and Human Behavior - 236667 - Winter 2024-2025</div>
<font size="6">Homework #4 - Recommendation Systems 🦘</font>


IDS: 316333368   314884602

# Instructions and submission guidelines

* Clone this notebook and complete the exercise:
    * Aim for clear and concise solutions.
    * Indicate clearly with a text block the sections of your solutions.
    * Answer dry questions in markdown blocks, and wet questions in code blocks.
* Submission guidelines:
    * Add a text block in the beginning of your notebook with your IDs.
    * When you're done, restart the notebook and make sure that everything runs smoothly (Runtime->"Restart and Run All")
    * Export your notebook as ipynb (File->Download->"Download .ipynb")
    * If you need to attach additional files to your submission (e.g images), add them to a zip file together with the notebook ipynb file.
    * Submit through the course website. Remember to list partner IDs when you submit.
* **Due date**: Sunday 12/01/2025, 10:00
* For any questions regarding this exercise, contact [Eden](mailto:edens@campus.technion.ac.il).

# Introduction

In workshops 4 and 5, we will be using a simulation framework for evaluating different recommendation algorithms. The framework consists of 3 main components - **environments** and **recommenders**, coupled together in the **simulation**. The design is inspired by the [RecLab](https://berkeley-reclab.github.io/) framework, introduced in [Krauth et al., 2020](https://arxiv.org/pdf/2011.07931).

In this homework assignment, you will implement a single step of the simulation to gain familiarity with the different components. The workshop will extend this code to examine how user behavior evolves over time.

## Notations

* Time is assumed to be discrete and denoted by $t\in\{0,1,\dots\}$
* The set of users is denoted by $U$.
* At each time $t$, the set of "online users" requesting recommendation is denoted by $U_t \subseteq U$.
* The set of all items is denoted by $X$.
* Rating given by user $u\in U$ to item $x \in X$ at time $t$ is denoted by $r_t(u,x)\in[1,5]$. User and item are not explicity specified when they are clear from context.
* Predicted ratings are denoted by $\hat{r}_t (u,x)$.
* Recommendations at time $t$ are denoted by $\{(u,x_u)\}_{u\in U_t}$.
* The Average Rating of Recommended Items (*ARRI*) at time $t$ is defined as:
$$
\mathrm{ARRI} =
\frac{1}{\left|U_t\right|}
\sum_{u\in U_t}
r_t (u, x_u)
$$
* Rating RMSE at time $t$ is defined as:
$$
\mathrm{RMSE} =
\left(
\frac{1}{\left|U_t\right|}
\sum_{u\in U_t}
\left(\hat{r}_t (u, x_u) - r_t (u, x_u) \right)^2
\right)^{0.5}
$$

## Environment
An *environment* defines the population of users and the collection of available items. It specifies the users' behavior, their preferences and the way they change over time, how the users rate items, etc. In particular, the environments we use here are **stateful**.

In this workshop, environments expose the following interface:

* `__init__(...)` - Initialize environment using given parameters
* `get_initial_ratings() -> padnas.DataFrame` - Returns a DataFrame with initial ratings $\{(u_i, x_i, r(u_i,x_i)\}$. Useful for bootstrapping the recommendation algorithm and avoiding the "[cold-start](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems))" problem.
* `get_online_users() -> numpy.array[int]` - Returns the set of online users $U_t\subseteq U$ that queried the system at the current time step $t$.
* `recommend(recommendations: List[tuple[int, int]]) -> pandas.DataFrame` - Receives as input a list of tuples $\{(u,x_u)\}_{u\in U_t}$, where each $u \in U_t$ is an online user, and $x_u$ is the item being recommended. Note that only unseen items can be recommended, and all online users must receive recommendations. The function returns the true ratings given by the users.


## Recommender

A *recommender* generates item recommendations to users based on past ratings.

In this workshop, prediction algorithms use the [Surprise](https://surpriselib.com/) framework, which provides implementations of common collaborative filtering algortihms.



## Simulation
The simulation works in discrete time steps $t\in\{1,2,\dots\}$. At each step $t$:
* The environent is queried for the current set of online users: $$U_t\subseteq U$$
* The recommender selects items based on predictions:
$$ \{(u,x_u)\}_{u\in U_t} $$
* The environment returns explicit feedback (true ratings) from the users, based on the given recommendations:
$$\{(u, x_u, r_t(u,x_u)\}_{u\in U_t}$$


## Imports

In [1]:
%pip install scikit-surprise

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import surprise

from tqdm.auto import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357271 sha256=0b9a365cce23d07d7dca978824a92033f3d336a5de837584d8552d25dde60120
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


# Homework task: Static environment

In this task we, will get ourselves familiar with the framework by exploring a static recommendation environment.


## The `TopicsStatic` environment

The following environment is based on the `topics-static` environment introduced by [Krauth et al., 2020](https://arxiv.org/abs/2011.07931).

* In the `TopicsStatic` environment, each item is assigned to one of $K$ topics and users prefer certain topics.
The preference of user $u$ for items of topic $k$
is initialized as $\pi(u, k) \sim \mathrm{Uniform}(0.5, 5.5)$, while the topic
$k_x$ of item $x$ is chosen uniformly at random from the set of all topics ($k_x\sim\mathrm{Uniform}([K])$).

* When user $u$ is recommended item $x $ of topic $k_x$, they will rate the item as:
$$r_t(u,x) = \mathrm{clip}_{[1,5]}\left(\pi(u, k_x) + \epsilon\right)$$
where $\epsilon \sim N(0, \sigma^2)$ represents exogenous noise not modeled by the simulation, and $\mathrm{clip}_{[1,5]}$ truncates values to be between $1$ and $5$.

## Environment implementation

Implementation of the TopicsStatic environment is given by the following class:

In [2]:
class TopicsStatic:
    topic_affinity_params = dict(
        low=0.5,
        high=5.5,
    )
    decision_noise_params = dict(
        scale=0.5,
    )
    rating_frequency = 0.2

    def __init__(self, n_users, n_items, n_topics, n_initial_ratings, random_state=None):
        """
        Initialize the environment.

        Parameters
        ----------
        n_users : int
            Number of users in the environment.

        n_items : int
            Number of items in the environment.

        n_topics : int
            Number of latent topics in the environment.

        n_initial_ratings : int
            Number of initial ratings available to the recommender before the
            simulation starts.

        random_state : int, default=None
            Random seed to use, if none is specified, a seed provided by the
            OS will be used.
        """
        self.n_users = n_users
        self.n_items = n_users
        self.all_users = [f'usr_{i}' for i in range(self.n_users)]
        self.all_items = [f'itm_{i}' for i in range(self.n_items)]
        self.n_topics = n_topics
        self.n_initial_ratings = n_initial_ratings
        self.rng = np.random.default_rng(random_state)
        # Assign topics to items
        self._item_topics = self.rng.integers(
            low=0,
            high=n_topics,
            size=n_items
        )
        # Initialize topics affinity matrix
        self._topic_affinity = self.rng.uniform(
            **self.topic_affinity_params,
            size=(n_users, n_topics),
        )
        # Initialize environment state
        self.t = 0
        self.last_online_users = None
        self.seen_items = {user_id: list() for user_id in self.all_users}
        self.initial_ratings_shown = False

    def _get_rating(self, user_id, item_id):
        """
        Calculate rating r_t(user_id, item_id).
        """
        assert item_id not in self.seen_items[user_id], (
            'Each item can only be shown once to each user'
        )
        user_internal_id = self.all_users.index(user_id)
        item_internal_id = self.all_items.index(item_id)
        self.seen_items[user_id].append(item_id)
        item_topic = self._item_topics[item_internal_id]
        affinity = self._topic_affinity[user_internal_id, item_topic]
        noise = self.rng.normal(**self.decision_noise_params)
        rating = np.clip(affinity+noise, 1, 5)
        return rating

    def get_initial_ratings(self):
        """
        Get initial ratings, to be used when initializing the recommender.

        Returns
        -------
        ratings_df : pandas.DataFrame
            DataFrame with columns: user_id, item_id, rating, timestamp.
            Timestamp for initial data is set to 0.
        """
        assert not self.initial_ratings_shown, (
            'Initial ratings can only be calculated once',
        )
        self.initial_ratings_shown = True
        all_pairs = list(itertools.product(
            self.all_users,
            self.all_items,
        ))
        selected_pairs = self.rng.choice(
            a=all_pairs,
            size=self.n_initial_ratings,
            replace=False,
        )
        ratings = [
            (user_id, item_id, self._get_rating(user_id, item_id))
            for user_id, item_id in selected_pairs
        ]
        return pd.DataFrame(
            data=ratings,
            columns=('user_id','item_id','rating')
        ).assign(timestamp=self.t)

    def get_online_users(self):
        """
        Returns the set of online users that queried the system at the
        current time step.

        Returns
        -------
        online_users: array of user_ids
        """
        assert self.last_online_users is None, (
            'Previous batch of online users must get recommendations'
        )
        n_online = self.rng.binomial(
            n=self.n_users,
            p=self.rating_frequency,
        )
        online_users = self.rng.choice(
            a=self.all_users,
            size=n_online,
            replace=False,
        )
        self.last_online_users = set(online_users)
        return online_users

    def recommend(self, recommendations):
        """
        Recommend items to users.

        Parameters
        ----------
        recommendations : list of (user_id, item_id) tuples.
            Each (user_id, item_id) tuple corresponds to an item recommended
            to an online user. Note that only unseen items can be recommended,
            and all online users must receive recommendations.

        Returns
        -------
        ratings_df : pandas.DataFrame
            True ratings given by the users.
            DataFrame with columns: user_id, item_id, rating, timestamp.
            Timestamp for recommendations is >= 1.
        """
        assert self.last_online_users is not None, (
            'Online users must be selected by calling get_online_users()'
        )
        assert len(recommendations)==len(self.last_online_users), (
            'Number of recommendations must match number of online users'
        )
        assert {user_id for user_id, _ in recommendations}==self.last_online_users, (
            'Users given recommendations must match online users'
        )
        assert all(item_id not in self.seen_items[user_id] for user_id, item_id in recommendations), (
            'Only unseen items can be recommended'
        )
        ratings = [
            (user_id, item_id, self._get_rating(user_id, item_id))
            for user_id, item_id in recommendations
        ]
        self.last_online_users = None
        self.t += 1
        return pd.DataFrame(
            data=ratings,
            columns=('user_id','item_id','rating')
        ).assign(timestamp=self.t)



## Initialize environment

We initialize the environment with the following code:

In [3]:
static_env = TopicsStatic(
    n_users=1000,
    n_items=1700,
    n_topics=19,
    n_initial_ratings=100000,
    random_state=1234,
)

Get initial ratings using `static_env.get_initial_ratings()`. Print the first 5 rows.

We will use this dataframe in the next steps. Note that you can only call `get_initial_ratings()` once for every initialization of the environment, so use a variable to save the result.

🔵 **Solution**:

In [4]:
## YOUR SOLUTION
initial_ratings = static_env.get_initial_ratings()
print(initial_ratings.head())

   user_id  item_id    rating  timestamp
0  usr_187  itm_749  1.000000          0
1  usr_830  itm_321  3.609729          0
2  usr_663  itm_171  1.577740          0
3  usr_115  itm_717  3.108401          0
4  usr_530   itm_89  3.425596          0


Get list of online users using `static_env.get_online_users()`. Print the results.

We will use this dataframe in the next steps. Note that you can only call `get_online_users()` function once for every simulation step of the environment, so use a variable to save the result.

🔵 **Solution**:

In [5]:
## YOUR SOLUTION
online_users = static_env.get_online_users()
print(online_users)

['usr_207' 'usr_527' 'usr_355' 'usr_647' 'usr_319' 'usr_670' 'usr_714'
 'usr_305' 'usr_793' 'usr_890' 'usr_916' 'usr_283' 'usr_469' 'usr_710'
 'usr_46' 'usr_676' 'usr_289' 'usr_18' 'usr_268' 'usr_808' 'usr_547'
 'usr_880' 'usr_480' 'usr_303' 'usr_628' 'usr_239' 'usr_278' 'usr_406'
 'usr_885' 'usr_748' 'usr_75' 'usr_533' 'usr_396' 'usr_165' 'usr_7'
 'usr_632' 'usr_284' 'usr_261' 'usr_999' 'usr_95' 'usr_629' 'usr_54'
 'usr_172' 'usr_795' 'usr_957' 'usr_200' 'usr_898' 'usr_909' 'usr_118'
 'usr_651' 'usr_285' 'usr_655' 'usr_302' 'usr_921' 'usr_570' 'usr_750'
 'usr_87' 'usr_184' 'usr_134' 'usr_317' 'usr_65' 'usr_747' 'usr_706'
 'usr_825' 'usr_232' 'usr_84' 'usr_77' 'usr_659' 'usr_144' 'usr_665'
 'usr_682' 'usr_796' 'usr_135' 'usr_878' 'usr_741' 'usr_449' 'usr_470'
 'usr_507' 'usr_241' 'usr_511' 'usr_273' 'usr_776' 'usr_623' 'usr_693'
 'usr_827' 'usr_454' 'usr_21' 'usr_259' 'usr_724' 'usr_901' 'usr_954'
 'usr_762' 'usr_617' 'usr_262' 'usr_504' 'usr_546' 'usr_865' 'usr_947'
 'usr_96' 'usr_324

## Train prediction model

[Surprise](https://surpriselib.com/) is a Python library for building and analyzing recommender systems that deal with explicit rating data.

In particular, Surprise provides various ready-to-use prediction algorithms such as baseline algorithms, neighborhood methods, matrix factorization-based, and many others.

1. Go over the introduction to Surprise in its [main homepage](https://surpriselib.com/) ("Overview", "Getting started" sections).
2. Go over the "[Basic usage](https://surprise.readthedocs.io/en/stable/getting_started.html#basic-usage)" section in the Surprise documentation.
3. The `surprise` library is already available in this notebook. (Surprise! 🥳)

Fit a [`surprise.SVD`](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD) predictor on the initial ratings returned by `static_env`. Use the default hyperparameters, and `1234` as the `random_state`.

Hint: The following function may be useful for converting a `pandas.DataFrame` to a `surprise.Trainset`:


In [6]:
def trainset_from_df(df):
    """
    Convert DataFrame to Surprise training set.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with columns [user_id, item_id, rating]

    Returns
    -------
    trainset : surprise.Trainset
    """
    dataset = surprise.Dataset.load_from_df(
        df=df[['user_id','item_id','rating']],
        reader=surprise.Reader(rating_scale=(1,5)),
    )
    return dataset.build_full_trainset()




🔵 **Solution**:

In [7]:
## YOUR SOLUTION
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset_from_df(initial_ratings))

## Recommend

Next, we use the trained model to obtain recommendations:
* Use the trained model to predict the ratings of all items, for all online users returned previously.
* Recommend to each user $u$ the unseen item $x$ with the highest predicted rating $\hat{r}(u,x)$. Output the results as a pandas DataFrame with columns `user_id`, `item_id`, `predicted_rating`. Display the first 5 rows.

Hint: Use the surprise `predict` function. Possible item ids are given by `static_env.all_items`. Items seen by user $u$ are stored in `static_env.seen_items[user_id]`.

🔵 **Solution**:

In [15]:
## YOUR SOLUTION

all_items = static_env.all_items
predictions = [
    max([algo.predict(user, item) for item in all_items if item not in static_env.seen_items[user]], key=lambda x: x.est)
    for user in online_users
]
df = pd.DataFrame([{'user_id': pred.uid, 'item_id': pred.iid, 'predicted_rating': pred.est} for pred in predictions])
print(df)

     user_id  item_id  predicted_rating
0    usr_207  itm_827          4.652473
1    usr_527  itm_207          5.000000
2    usr_355  itm_866          4.795553
3    usr_647  itm_348          4.828278
4    usr_319  itm_541          4.353651
..       ...      ...               ...
189  usr_187  itm_780          4.619321
190  usr_704    itm_8          4.512322
191  usr_256  itm_369          4.951711
192  usr_516  itm_532          4.754541
193   usr_63  itm_704          4.934853

[194 rows x 3 columns]


Use the dataframe to construct a list of recommendations tuples `(user_id, item_id)`, and recommend them to the users. Save the feedback in a variable.

🔵 **Solution**:

In [16]:
## YOUR SOLUTION
recommendations = [(pred.uid, pred.iid) for pred in predictions]
print(recommendations)

[('usr_207', 'itm_827'), ('usr_527', 'itm_207'), ('usr_355', 'itm_866'), ('usr_647', 'itm_348'), ('usr_319', 'itm_541'), ('usr_670', 'itm_352'), ('usr_714', 'itm_350'), ('usr_305', 'itm_193'), ('usr_793', 'itm_105'), ('usr_890', 'itm_506'), ('usr_916', 'itm_250'), ('usr_283', 'itm_640'), ('usr_469', 'itm_85'), ('usr_710', 'itm_528'), ('usr_46', 'itm_89'), ('usr_676', 'itm_95'), ('usr_289', 'itm_125'), ('usr_18', 'itm_815'), ('usr_268', 'itm_343'), ('usr_808', 'itm_850'), ('usr_547', 'itm_270'), ('usr_880', 'itm_922'), ('usr_480', 'itm_128'), ('usr_303', 'itm_424'), ('usr_628', 'itm_548'), ('usr_239', 'itm_651'), ('usr_278', 'itm_90'), ('usr_406', 'itm_940'), ('usr_885', 'itm_341'), ('usr_748', 'itm_985'), ('usr_75', 'itm_482'), ('usr_533', 'itm_748'), ('usr_396', 'itm_848'), ('usr_165', 'itm_806'), ('usr_7', 'itm_568'), ('usr_632', 'itm_548'), ('usr_284', 'itm_65'), ('usr_261', 'itm_283'), ('usr_999', 'itm_971'), ('usr_95', 'itm_673'), ('usr_629', 'itm_379'), ('usr_54', 'itm_178'), ('u

## Evaluate

Calculate the ARRI and RMSE for the given recommendations, and print the results.

Hint: Definitions for ARRI and RMSE can be found in the "Technical Preliminaries" section above. The [`pandas.merge(...)`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge) function may be useful for joining between rating predictions and user feedback.

🔵 **Solution**:

In [18]:
## YOUR SOLUTION
users_df = static_env.recommend(recommendations)
items_and_rec = pd.merge(df, users_df, how="left", on="user_id")
print(items_and_rec)

     user_id item_id_x  predicted_rating item_id_y    rating  timestamp
0    usr_207   itm_827          4.652473   itm_827  4.961160          1
1    usr_527   itm_207          5.000000   itm_207  4.844150          1
2    usr_355   itm_866          4.795553   itm_866  4.436802          1
3    usr_647   itm_348          4.828278   itm_348  3.543195          1
4    usr_319   itm_541          4.353651   itm_541  5.000000          1
..       ...       ...               ...       ...       ...        ...
189  usr_187   itm_780          4.619321   itm_780  5.000000          1
190  usr_704     itm_8          4.512322     itm_8  5.000000          1
191  usr_256   itm_369          4.951711   itm_369  4.716882          1
192  usr_516   itm_532          4.754541   itm_532  5.000000          1
193   usr_63   itm_704          4.934853   itm_704  2.959604          1

[194 rows x 6 columns]


In [19]:
mse = np.mean((np.array(items_and_rec.predicted_rating) - np.array(items_and_rec.rating)) ** 2)
rmse = np.sqrt(mse)
print(rmse)

0.747878638861323


In [20]:
arri = np.mean(np.array(items_and_rec.rating))
print(arri)

4.398460424753723
